In [ ]:
import time,serial,random
import numpy as np
import scipy,pybrain3

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=1):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':pass
    acc = [float(e) for e in ser.readline().split(b'\t')]
    print(acc)
    if fall_check(*acc)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print("servo in: ",a,b,c,d,e,f,delay)
    else:
         1/0
    return acc
    
def fall_check(accelx,accely,accelz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns FALSE if Robo - NOT fallen
        returns TRUE if Robo - has fallen'''
    if (abs(accelx)<5 or abs(accelz-2)<5) and accely>7:return False
    else: return True
    
def interpreter(st,mtrx):
    '''gets input string and last value of servoin
        returns tuple of updated values servo
        if str='start' to initials
        if str='a=12 c=13' returns (12,90,13,90,90,90,1)
        if str='(12,90,13,90,90,90,1)' returns the same
        if str='(12,90,13,90)' returns the same
        if str='a=12 c=13 then e=50 then f=40 then start' compile the algorythm
        if str='a=5 c=3 then while a=6 b=2 and a=4 b=4' compile
        if str='a=5 c=3 then wait 3 then b=5' compile
    '''
    old=[*mtrx[-1]]
    if st=='start':
        mtrx = np.vstack([mtrx,[90,90,90,90,90,90,1]])
        return mtrx
    
    if st=='jackson':
        mtrx = interpreter('start then for 8 b=120 d=120 and a=120 c=120 and wait 1 and a=60 c=60 and b=60 d=60',mtrx)
        return mtrx
    
    if st=='walk':
        mtrx = interpreter('start then for 1 e=80 g=3 and f=80 b=70 a=70 and a=50 and d=70 f=90 e=90 c=70 and e=110 f=110 b=100 a=90 d=90 and c=100 e=120 b=120 g=1 and b=130 d=110 a=120 c=120 and c=140 e=90 f=90 g=3 and e=80 f=80 d=80 b=90 a=60 c=80',mtrx)
        return mtrx
    
    if st[0:4]=='wait':
        time.sleep(int(st[5]))
        mtrx=interpreter(st[11:],mtrx)
        return mtrx
    
    if st[0:5]=='while':
        while True:
            for j in st[6:].split(' and '):
                mtrx=interpreter(j,mtrx)
    
    if st[0:3]=='for':
        for l in range(int(st[4])): 
            for j in st[6:].split(' and '):
                mtrx=interpreter(j,mtrx)
        return mtrx
    
    try: #12,13,15
        q = list(int(i) for  i in st.split(','))
        for j in range(len(q),7): q.append(old[j])
        old = [q[k] for k in range(len(q))]
        mtrx = np.vstack([mtrx,old])
        return mtrx
    except ValueError:
        try: #a=12 b=13
            q = old
            tmp = list(st.split())
            for i in tmp: 
                q[ord(str(i).split('=')[0])-97]=int(str(i).split('=')[1])
            old = [q[k] for k in range(len(q))]
            mtrx = np.vstack([mtrx,old])
            return mtrx
        except IndexError: #a=12 b=13 then b=15
            for j in st.split(' then '):
                mtrx = interpreter(str(j),mtrx)
            return mtrx
        
def serial_begin(port):
    ser = serial.Serial(port, 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    print("connected to: "+ ser.portstr)
    return ser,np.array([[90,90,90,90,90,90,1]])

def executor(ser,prt,mtrx):
    k = 1
    try:#fall detection script            
        for i in mtrx: 
            servoin(*i)
            k += 1
    except ZeroDivisionError:
        print('I have fallen.')
        stand_up([90,90,90,90,90,90,1])
    return ser,k

def stand_up(old):
    """TRUE when ready"""
    try:
        data='in'
        for i in old:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        servoin(*old)
        print('Back in service.')
    except ZeroDivisionError:
        print('Master, lift me up, please...')
        stand_up(old)
        

def neural_network_solution(mtrx,steps_to_fall):
    '''change mtrx throu algorithm according to steps_to_fall'''
    return mtrx

def smart_sum(a,b):
    '''returns a sum of matrixes not crossed a borders'''
    borders=[[10,170],[10,170],[10,170],[10,170],[10,170],[10,170],[0,30]]
    c = np.array(a)
    for i in range(len(a)):
        for j in range(len(a[0])):
            if borders[j][1]>a[i][j]+b[i][j]>borders[j][0]:
                c[i][j] = a[i][j]+b[i][j]
    return c
    
if __name__ == "__main__":
    port = 'COM5'
    ser,matrix = serial_begin(port)
    while True:
        stand_up(*[[*matrix[-1]]])
        inp = input()
        if inp=="exit":break
        if inp[0:5]=="learn": #learn case
            matrix = interpreter(inp[6:],[[*matrix[-1]]]) #intepreter to matrix all params after 'learn'
            print('Interpreted as a beginning of learn cycle.')
            ser,stf = executor(ser,port,matrix)
            best = stf
            while True:
                random_matrix = np.random.randint(-1,2,size=(matrix.shape))
                tmp = smart_sum(matrix, random_matrix)
                ser,stf = executor(ser,port,tmp)
                if stf>best:
                    matrix = tmp
                    best = stf 
                    print('Found a better way.')
                stand_up([90,90,90,90,90,90,1])
        else: #classic linear logic
            matrix = interpreter(inp,[[*matrix[-1]]])
            steps = len(matrix)
            ser,stf = executor(ser,port,matrix[1:])
            if stf<steps: print("Steps to fall :", stf)
                    
                    
                    
                    
                    

In [ ]:
ser.close()